## Load VIIRS file(s)

In [34]:
import tifffile as tiff
import numpy as np
#from PIL import Image
#Image.MAX_IMAGE_PIXELS = 1000000000
import matplotlib.pyplot as plt
from osgeo import gdal
from osgeo_utils import gdal_merge
gdal.UseExceptions()

In [35]:
viirs_tower_path = "/home/mxh/VIIRS_DEC2023_GERITAFRA/"

#modis_volume_path="/media/mxh/Volume/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"
#modis_win_path = "F:\\Bachelorarbeit\\sat-data\\modis\\MODIS_MOD09GA_GERITAFRA_31day_DEC2023\\"
#modis_x_path = "/mnt/f/Bachelorarbeit/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"
viirs_path = viirs_tower_path

# Definitionen

In [ ]:
# todo upscale M-Bands to 500x500m or get different product

In [36]:
def merge_VIIRS_refl_bands_to_tif(path, filename, size=2400, extension=".h5"):
    """
    filename without .hdf file extension!
    """
    viirs_in_modis_order = ["I1", "I2", "M3", "M4", "M8", "M10", "M11"]
    band_drivers = { # these are in modis order
    "I1": f'HDF5:"{viirs_path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SurfReflect_I1_1',
    "I2": f'HDF5:"{viirs_path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SurfReflect_I2_1',
    "M3" : f'HDF5:"{viirs_path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SurfReflect_M3_1',
    "M4": f'HDF5:"{viirs_path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SurfReflect_M4_1',
    "M8": f'HDF5:"{viirs_path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SurfReflect_M8_1',
    "M10": f'HDF5:"{viirs_path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SurfReflect_M10_1',
    "M11": f'HDF5:"{viirs_path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SurfReflect_M11_1',
    }


    #f= gdal.Open(path+filename+extension)
    #subsets = f.GetSubDatasets()
    #driver = subsets[11][0][:-3] # get common driver command # cant i just loop over drivers??
    # TODO: Reorder VIIRS bands to fit the L8 order. The model is trained on this order.
    
    L8_order = [3, 4, 1, 2, 6, 7, 5] # [2, 3, 0, 1, 5, 6, 4] # landsat8 order of modis (modga09) bands
    # remember to +1 every band as MODIS starts its bands at index 1
    for i in L8_order:  # translate bands 1-7 to geotiff
        driveri = band_drivers[i-1]
        band = gdal.Open(driveri)
        gdal.Translate(destName=f'{viirs_path+filename}_b{i}_{size}.tiff', srcDS=band, format="GTiff", width=size, height=size)
    
    #gdal.Warp(destNameOrDestDS=path+filename+'_warped.tif', srcDSOrSrcDSTab=[f'{path+filename}_b{i}.tif' for i in range(1,7+1)])
    
    # merge_params = ['', '-separate']+[f'{path+filename}_b{i}.tif' for i in range(1,7+1)]+['-o', path+filename+'_merged.tif']
    merge_params = ['', '-separate']+[f'{path+filename}_b{i}_{size}.tiff' for i in L8_order]+['-o', path+filename+f'_merged_{size}.tiff']
    gdal_merge.main(merge_params)
    #!{gdal_path}gdal_merge.py -separate {modis_path+day1}_b[1-7].tif -o {modis_path+day1}_merged.tif

In [37]:
def warp_VIIRS_to_epsg(path, filename, postfix="_wgs84_warped", SRS="EPSG:4326", translate=True):
    """
    filename: without .tiff extension
    """
    warp_options = gdal.WarpOptions(dstSRS=SRS)
    warp = gdal.Warp(destNameOrDestDS=path+filename+postfix+".tmp.tiff", srcDSOrSrcDSTab=path+filename+".tiff", options=warp_options)
    if translate:
        gdal.Translate(destName=path+filename+postfix+".tiff", srcDS=warp, format="GTiff") # , creationOptions=['TILED:YES']
    warp = None

In [38]:
def upscale_single_layer(path, filename, name, driver, size=2400):
    # upscale gdal
    translate_opts = {"format": "GTiff", "resampleAlg": "nearest", "width": size, "height": size} #,"creationOptions": ['TILED:YES']}
    # resampleAlgs -r {nearest|bilinear|cubic|cubicspline|lanczos|average|rms|mode}
    gdal.Translate(path + filename + f"_{name}_{size}.tiff", driver, **translate_opts)

In [39]:
def upscale_all_angle_layers(path, filename, size=2400):
    # need to be sorted like so:
    #HDF5:"/home/mxh/VIIRS_DEC2023_GERITAFRA/VNP09GA.A2023339.h18v04.002.2023340101055.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SensorAzimuth_1
    drivers = {"solar_zenith": f'HDF5:"{path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SolarZenith_1',
               "solar_azimuth": f'HDF5:"{path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SolarAzimuth_1',
               "sensor_zenith": f'HDF5:"{path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SensorZenith_1',
               "sensor_azimuth": f'HDF5:"{path}{filename}.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SensorAzimuth_1'}

    for k in drivers.keys():
        upscale_single_layer(path, filename, name=k, driver=drivers[k], size=size)

    merge_params = ['', '-separate']+[(path + filename + f"_{name}_{size}.tiff") for name in drivers.keys()]+['-o', path+filename+f'_upscaled_angles_{size}.tiff']
    gdal_merge.main(merge_params)

# Do Prep for multiple defined days:

#### Steps
1.1 Mergen zu Merged1-7, dabei Projection + Geometry beibehalten

1.2 Warpen zu _wgs84.tif

2.1 Upscalen der Solar/Sensor Zenith/Azimuth Layers zu 2400x2400 bzw. um Faktor 2,  dabei Projection + Geometry beibehalten

2.2 Warpen zu _wgs84.tif

3. Siehe sis_RunFmaskOnMODIS.ipynb

3.1 Input der Dateien und Bänder-Zuordnung in Fmask Config Objekte

3.2 Run Fmask

4 Upsample der State1km Datei und rewarp zu wgs84

In [12]:

curr_size=2400

In [13]:
import os

In [26]:
h5_dir = os.listdir(viirs_path)
h5_dir = [f[:-3] for f in h5_dir if f[-3:] == '.h5']
days = {}
for i, v in enumerate(h5_dir):
    days[i] = v

In [27]:
days

{0: 'VNP09GA.A2023339.h18v04.002.2023340101055',
 1: 'VNP09GA.A2023335.h18v04.002.2023336092641',
 2: 'VNP09GA.A2023343.h18v04.002.2023344084949',
 3: 'VNP09GA.A2023346.h18v04.002.2023347221456',
 4: 'VNP09GA.A2023346.h18v04.002.2023347150821',
 5: 'VNP09GA.A2023355.h18v04.002.2023356100008',
 6: 'VNP09GA.A2023356.h18v04.002.2023357091706',
 7: 'VNP09GA.A2023348.h18v04.002.2023349082521',
 8: 'VNP09GA.A2023340.h18v04.002.2023341095423',
 9: 'VNP09GA.A2023351.h18v04.002.2023352101136',
 10: 'VNP09GA.A2023364.h18v04.002.2023365083229',
 11: 'VNP09GA.A2023357.h18v04.002.2023358085917',
 12: 'VNP09GA.A2023353.h18v04.002.2023354084105',
 13: 'VNP09GA.A2023345.h18v04.002.2023346093914',
 14: 'VNP09GA.A2023337.h18v04.002.2023338085057',
 15: 'VNP09GA.A2023341.h18v04.002.2023342090252',
 16: 'VNP09GA.A2023338.h18v04.002.2023339083613',
 17: 'VNP09GA.A2023365.h18v04.002.2024006085613',
 18: 'VNP09GA.A2023362.h18v04.002.2023363102430',
 19: 'VNP09GA.A2023358.h18v04.002.2023359083915',
 20: 'VNP0

In [40]:
for i, (k, filename) in enumerate(days.items()):
    day = filename# filename[:-4] # without file extension
    #step 1
    merge_VIIRS_refl_bands_to_tif(viirs_path, day, size=curr_size)
    warp_VIIRS_to_epsg(viirs_path, day+f"_merged_{curr_size}")
    
    #step 2
    upscale_all_angle_layers(viirs_path, day, size=curr_size)
    
    warp_VIIRS_to_epsg(viirs_path, filename=day+f"_upscaled_angles_{curr_size}")
    angles = ["solar_zenith", "solar_azimuth", "sensor_zenith", "sensor_azimuth"]
    for a in angles:  # also warp all individual angle files, for Fmask AnglesFileInfo input
        warp_VIIRS_to_epsg(viirs_path, filename=day+"_"+a+"_"+str(curr_size))
    
    #step 3
    # done in fmask notebook

    #TODO: step 4
    #driver = f'HDF5:"{viirs_path}{day}.hdf":VIIRS_Grid_1km_2D:state_1km_1'
    #upscale_single_layer(viirs_path, day, name="state_1km_1", driver=driver, size=curr_size)
    #warp_VIIRS_to_epsg(viirs_path, day+f"_state_1km_1_{curr_size}")

RuntimeError: HDF5:"/home/mxh/VIIRS_DEC2023_GERITAFRA/VNP09GA.A2023339.h18v04.002.2023340101055.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SurfReflect_M6_1: No such file or directory

# TEST VIIRS

In [31]:
path = viirs_path
filename = "VNP09GA.A2023339.h18v04.002.2023340101055" 
extension = ".h5"

In [32]:
f= gdal.Open(path+filename+extension)
subsets = f.GetSubDatasets()

https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/s-npp-nasa-viirs-overview/#viirs-spectral-bands
<pre>
Band 	Reflected Range (μm) 	Band Explanation
I1 	0.6 - 0.68 	Visible/Reflective
I2 	0.85 - 0.88 	Near Infrared
I3 	1.58 - 1.64 	Shortwave Infrared
I4 	3.55 - 3.93 	Medium-wave Infrared
I5 	10.5 - 12.4 	Longwave Infrared
M1 	0.402 - 0.422 	Visible/Reflective
M2 	0.436 - 0.454 	Visible/Reflective
M3 	0.478 - 0.488 	Visible/Reflective
M4 	0.545 - 0.565 	Visible/Reflective
M5 	0.662 - 0.682 	Near Infrared
M6 	0.739 - 0.754 	Near Infrared
M7 	0.846 - 0.885 	Shortwave Infrared
M8 	1.23 - 1.25 	Shortwave Infrared
M9 	1.371 - 1.386 	Shortwave Infrared
M10 	1.58 - 1.64 	Shortwave Infrared
M11 	2.23 - 2.28 	Medium-wave Infrared
M12 	3.61 - 3.79 	Medium-wave Infrared
M13 	3.97 - 4.13 	Longwave Infrared
M14 	8.4 - 8.7 	Longwave Infrared
M15 	10.26 - 11.26 	Longwave Infrared
M16 	11.54 - 12.49 	Day/Night
DNB 	0.5 - 0.9 	Visible/Reflective 
</pre>

In [ ]:
# red, nir, blue, green, cirrus, swir1, swir2
modis_order = ["I1", "I2", "M3", "M4", "M8", "M10", "M11"]

In [42]:
f.GetMetadata()

{'AdditionalLayers1KM': '3',
 'AdditionalLayers500M': '1',
 'AlgorithmType': 'SCI',
 'AlgorithmVersion': 'NPP_PR09G 2.0.0',
 'AutomaticQualityFlagExplanation': 'No automatic quality assessment is performed in the PGE',
 'CharacteristicBinAngularSize1KM': '30',
 'CharacteristicBinAngularSize500M': '15',
 'CharacteristicBinSize1KM': '926.625433055556',
 'CharacteristicBinSize500M': '463.312716527778',
 'Conventions': 'CF-1.6',
 'CoverageCalculationMethod': 'volume',
 'CoverageMinimum': '0.00999999977648258',
 'creator_email': 'modis-ops@lists.nasa.gov',
 'creator_name': 'VIIRS Land SIPS Processing Group',
 'creator_url': 'https://ladsweb.modaps.eosdis.nasa.gov',
 'DataColumns1KM': '1200',
 'DataColumns500M': '2400',
 'DataResolution': '1km and 500m',
 'DataRows1KM': '1200',
 'DataRows500M': '2400',
 'DayNightFlag': 'Terminator',
 'DeepOceanFlag': 'Yes',
 'EastBoundingCoord': '15.5702026309755',
 'EndTime': '2023-12-05 23:59:59',
 'FirstLayerSelectionCriteria': 'order of input pointer',
 

In [33]:
subsets

[('HDF5:"/home/mxh/VIIRS_DEC2023_GERITAFRA/VNP09GA.A2023339.h18v04.002.2023340101055.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SensorAzimuth_1',
  '[1200x1200] //HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SensorAzimuth_1 (16-bit integer)'),
 ('HDF5:"/home/mxh/VIIRS_DEC2023_GERITAFRA/VNP09GA.A2023339.h18v04.002.2023340101055.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SensorZenith_1',
  '[1200x1200] //HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SensorZenith_1 (16-bit integer)'),
 ('HDF5:"/home/mxh/VIIRS_DEC2023_GERITAFRA/VNP09GA.A2023339.h18v04.002.2023340101055.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SolarAzimuth_1',
  '[1200x1200] //HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SolarAzimuth_1 (16-bit integer)'),
 ('HDF5:"/home/mxh/VIIRS_DEC2023_GERITAFRA/VNP09GA.A2023339.h18v04.002.2023340101055.h5"://HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SolarZenith_1',
  '[1200x1200] //HDFEOS/GRIDS/VIIRS_Grid_1km_2D/Data_Fields/SolarZenith_1 (16-bit integer)'),
 ('HDF5:"/home/m